In [1]:
#!pip install fake-useragent
!pip install fake_useragent
import requests
from fake_useragent import UserAgent
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.5 MB/s eta 0:00:00


In [2]:
import random

def generate_custom_headers():
    accept_options = [
    "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "text/html,application/json;q=0.9,*/*;q=0.8",
    "text/html,application/xhtml+xml;q=0.9,*/*;q=0.8"
    ]


    accept_language_options = [
        "en-US,en;q=0.9",
        "en-GB,en;q=0.8",
        "en;q=0.7",
        "en-US,en;q=0.9,fr;q=0.8",
    ]

    accept_encoding_options = [
        "gzip, deflate, br",
        "gzip, deflate",
    ]

    referer_options = [
    "https://www.google.com/",
    "https://www.bing.com/",
    "https://news.google.com/",
    "https://www.yahoo.com/",
    "https://duckduckgo.com/",
    "https://www.facebook.com/",
    "https://twitter.com/",
    "https://www.reddit.com/",
    "https://www.youtube.com/",
    ]


    upgrade_insecure_requests_options = [
        "1",
    ]

    sec_fetch_mode_options = [
        "navigate"
    ]

    headers = {
        "User-Agent" : agent.random,
        "Accept": random.choice(accept_options),
        "Accept-Language": random.choice(accept_language_options),
        "Accept-Encoding": random.choice(accept_encoding_options),
        "Connection": 'keep-alive',
        "Referer": random.choice(referer_options),
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": random.choice(sec_fetch_mode_options),
        "Sec-Fetch-Site": "cross-site",
        "Sec-Fetch-User": "?1",
    }

    return headers
agent = UserAgent()

In [5]:
generate_custom_headers()

{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
 'Accept': 'text/html,application/json;q=0.9,*/*;q=0.8',
 'Accept-Language': 'en-US,en;q=0.9',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'Referer': 'https://www.youtube.com/',
 'Upgrade-Insecure-Requests': '1',
 'Sec-Fetch-Dest': 'document',
 'Sec-Fetch-Mode': 'navigate',
 'Sec-Fetch-Site': 'cross-site',
 'Sec-Fetch-User': '?1'}

In [6]:
import requests
import xml.etree.ElementTree as ET
import time
import json
from tqdm import tqdm  # For progress bar

def harvest_arxiv_metadata(domain="econ", from_date="2017-05-11", until_date="2025-04-15", max_batches=1):
    base_url = "https://export.arxiv.org/oai2"
    metadata_prefix = "arXiv"
    all_records = []
    token = None
    delay = 5  # seconds between requests to be polite
    batch_count = 0

    while batch_count < max_batches:
        if token:
            params = {
                "verb": "ListRecords",
                "resumptionToken": token
            }
        else:
            params = {
                "verb": "ListRecords",
                "set": domain,
                "from": from_date,
                "until": until_date,
                "metadataPrefix": metadata_prefix
            }

        print(f"Fetching batch {batch_count + 1}...")

        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break

        try:
            root = ET.fromstring(response.content)
        except ET.ParseError as e:
            print(f"Error parsing XML: {e}")
            break

        records = root.findall(".//{http://www.openarchives.org/OAI/2.0/}record")

        for record in records:
            metadata = record.find("{http://www.openarchives.org/OAI/2.0/}metadata")
            if metadata is not None:
                arxiv_meta = metadata.find("{http://arxiv.org/OAI/arXiv/}arXiv")
                if arxiv_meta is not None:
                    record_data = {
                        "id": arxiv_meta.findtext("{http://arxiv.org/OAI/arXiv/}id"),
                        "title": arxiv_meta.findtext("{http://arxiv.org/OAI/arXiv/}title"),
                        "authors": [a.findtext("{http://arxiv.org/OAI/arXiv/}keyname") for a in arxiv_meta.findall("{http://arxiv.org/OAI/arXiv/}authors/{http://arxiv.org/OAI/arXiv/}author")],
                        "categories": arxiv_meta.findtext("{http://arxiv.org/OAI/arXiv/}categories"),
                        "abstract": arxiv_meta.findtext("{http://arxiv.org/OAI/arXiv/}abstract"),
                        "created": arxiv_meta.findtext("{http://arxiv.org/OAI/arXiv/}created"),
                    }
                    all_records.append(record_data)

        # Get resumption token
        token_elem = root.find(".//{http://www.openarchives.org/OAI/2.0/}resumptionToken")
        token = token_elem.text.strip() if token_elem is not None and token_elem.text else None

        if not token:
            print("All records fetched.")
            break

        time.sleep(delay)
        batch_count += 1
        tqdm.write(f"Batch {batch_count} processed...")

    print(f"Total records fetched: {len(all_records)}")

    # Save to JSON
    output_filename = f"arxiv_{domain}_metadata.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(all_records, f, indent=2, ensure_ascii=False)

    print(f"Metadata saved to {output_filename}")


In [ ]:
# Code     | Discipline
# cs       | Computer Science
# math     | Mathematics
# stat     | Statistics
# eess     | Electrical Engineering and Systems Science
# econ     | Economics
# q-fin    | Quantitative Finance
# physics  | Physics (umbrella for multiple subfields)
# astro-ph | Astrophysics
# cond-mat | Condensed Matter Physics
# gr-qc    | General Relativity and Quantum Cosmology
# hep-ex   | High Energy Physics – Experiment
# hep-lat  | High Energy Physics – Lattice
# hep-ph   | High Energy Physics – Phenomenology
# hep-th   | High Energy Physics – Theory
# math-ph  | Mathematical Physics
# nlin     | Nonlinear Sciences
# nucl-ex  | Nuclear Experiment
# nucl-th  | Nuclear Theory
# quant-ph | Quantum Physics

# put the code you want in domain feild
# start_date and end date will be u decide it these date start_date must on which date the Discipline is created we can not use date before the creation of Discipline date
# max_batchs =  1 (default) here you should select the how many batches you want i batch consists of 1000 papers

In [7]:
harvest_arxiv_metadata(max_batches=1) # you can select any number according to your time because it takes few hours to complete the section like cs

Fetching batch 1...
Batch 1 processed...
Total records fetched: 1000
Metadata saved to arxiv_econ_metadata.json


In [9]:
import pandas as pd

df = pd.read_json("/content/arxiv_econ_metadata.json")
df.to_csv("arxiv_econ_metadata.csv", index=False)

,id,title,authors,categories,abstract,created
0,802.33000,Projective Expected Utility,['La Mura'],quant-ph cs.GT econ.TH,Motivated by several classic decision-theore...,2008-02-22
1,1107.02370,Team Decision Problems with Classical and Quan...,"['Brandenburger', 'La Mura']",quant-ph cs.IT econ.TH math-ph math.IT math.MP,We study team decision problems where commun...,2011-07-01
2,1409.17860,Zero-determinant strategies in iterated multi-...,['Guo'],cs.GT econ.GN physics.soc-ph q-fin.EC,"Self-serving, rational agents sometimes coop...",2014-09-05
3,1411.62500,Identification and Estimation of Multidimensio...,"['Aryal', 'Zincenko']",econ.GN q-fin.EC,We study the identification and estimation o...,2014-11-23
4,1506.00348,Enhanced Gravity Model of trade: reconciling m...,"['Almog', 'Bird', 'Garlaschelli']",physics.soc-ph cond-mat.dis-nn econ.GN q-fin.EC,The structure of the International Trade Net...,2015-06-01
...,...,...,...,...,...,...
995,2211.03002,Projecting XRP price burst by correlation tens...,"['Chakraborty', 'Hatsuda', 'Ikeda']",physics.soc-ph econ.GN q-fin.EC,Cryptoassets are becoming essential in the d...,2022-11-05
996,2211.03302,Optimal Scoring Rules for Multi-dimensional Ef...,"['Hartline', 'Shan', 'Li', 'Wu']",cs.GT econ.TH,This paper develops a framework for the desi...,2022-11-06
997,2211.04027,Bootstraps for Dynamic Panel Threshold Models,"['Gong', 'Seo']",econ.EM,This paper develops valid bootstrap inferenc...,2022-11-08
998,2211.04388,Profit Shifting and International Tax Reforms,"['Ferrari', 'Laffitte', 'Parenti', 'Toubal']",econ.GN q-fin.EC,"International taxation rules are outdated, a...",2022-11-08
